### This notebook will be mainly used for the capstone project.

In [2]:
import pandas as pd
import numpy as np
print ('Hello Capstone Project Course!')

Hello Capstone Project Course!


### Week 3 Assignment 1

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # used to get latitude and longitude values

import requests 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

#Scrape the wikipedia page and remove all 'not assigned' boroughs and assign all not assigned neighborhoods to the name of its respective borough.
#Create a dataframe for the toronto downtown area for kmeans cluster.

In [4]:
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header=0)[0]

df = df[df.Borough != "Not assigned"]
toronto_cluster =df[df.Borough == "Downtown Toronto"]

df["Neighbourhood"].replace("Not assigned","Queen's Park", inplace=True)

##Group all neighborhoods with the same postal code

In [17]:
comma = lambda a: " , ".join(a) 

df = df.groupby(['Postcode','Borough'], sort = False).agg(comma)
df

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront , Regent Park"
M6A,North York,"Lawrence Heights , Lawrence Manor"
M7A,Queen's Park,Queen's Park
M9A,Etobicoke,Islington Avenue
M1B,Scarborough,"Rouge , Malvern"
M3B,North York,Don Mills North
M4B,East York,"Woodbine Gardens , Parkview Hill"


##Calculate the shape of the dataframe

In [10]:
df.shape

(103, 1)

### Week 3 Assignment 2

In [9]:
df1=pd.read_csv("http://cocl.us/Geospatial_data")
df1 = df1.rename(columns={'Postal Code': 'Postcode'})
#Assign each postal code with its respective latitude and longitude
df2 = df.join(df1.set_index('Postcode'), on='Postcode')
df2.head()

,,Neighbourhood,Latitude,Longitude
Postcode,Borough,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Harbourfront , Regent Park",43.654260,-79.360636
M6A,North York,"Lawrence Heights , Lawrence Manor",43.718518,-79.464763
M7A,Queen's Park,Queen's Park,43.662301,-79.389494


### Week 3 Assignment 3

In [12]:
#cluster toronto neighborhoods with a kcluster factor of 5.

toronto_cluster_merge = pd.merge(toronto_cluster,df1, on='Postcode')
toronto_cluster_merge = toronto_cluster_merge[toronto_cluster_merge.Neighbourhood != "St. James Town"]
kclusters = 5

toronto_cluster_loc = toronto_cluster_merge[["Latitude", "Longitude"]]

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_cluster_loc)

kmeans.labels_.shape

columns = ['Neighbourhood']
neighborhoods_venues_sorted = pd.DataFrame(columns=['Neighbourhood'])

neighborhoods_venues_sorted['Neighbourhood'] = toronto_cluster_merge['Neighbourhood']

neighborhoods_venues_sorted.drop_duplicates(keep=False,inplace=True) 
neighborhoods_venues_sorted.reset_index()
neighborhoods_venues_sorted.shape

(35, 1)

In [14]:
#Add the cluster labels to the toronto area dataframe
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_cluster_merge = toronto_cluster_merge.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_cluster_merge

ValueError: cannot insert Cluster Labels, already exists

In [15]:
#Create a map of the toronto area which shows the clusters around the downtown area.
map = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_cluster_merge['Latitude'], toronto_cluster_merge['Longitude'], toronto_cluster_merge['Neighbourhood'], toronto_cluster_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map)
       
map